In [29]:
import pandas as pd

In [30]:
df = pd.read_csv("en openfoodfacts org products.csv")

In [31]:
df.drop(df[df['pnns_groups_1'] == "unknown"].index, inplace=True)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4735 entries, 6971 to 13218
Data columns (total 18 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   code                                     4735 non-null   int64  
 1   product_name                             4475 non-null   object 
 2   quantity                                 3582 non-null   object 
 3   packaging_en                             1798 non-null   object 
 4   brands_en                                4488 non-null   object 
 5   categories_en                            4653 non-null   object 
 6   origins_en                               888 non-null    object 
 7   labels_en                                2494 non-null   object 
 8   countries_en                             4735 non-null   object 
 9   nutriscore_grade                         4650 non-null   object 
 10  pnns_groups_1                            4650 non

In [33]:
df.dropna(subset=['pnns_groups_2'], inplace=True)

In [34]:
df.to_csv("cleaned_data.csv")

In [35]:
df_co2 = pd.read_csv('https://raw.githubusercontent.com/Phi-project3/finalproject/refs/heads/main/dossier/donnees/source/category_schema/products_category_schema_v1.csv', sep=';')

df_co2.dropna(subset=['category_name'], inplace=True)

df_merged = df.merge(df_co2[['category_name', 'average']], how='left', left_on='pnns_groups_2', right_on='category_name')

df_merged.drop(columns=['category_name', 'carbon-footprint_100g', 'carbon-footprint-from-meat-or-fish_100g'], inplace=True)

df_merged.rename(columns={'average': 'carbon-footprint_1kg'}, inplace=True)

df_merged.sample(5)

,code,product_name,quantity,packaging_en,brands_en,categories_en,origins_en,labels_en,countries_en,nutriscore_grade,pnns_groups_1,pnns_groups_2,food_groups_en,environmental_score_score,environmental_score_grade,main_category_en,carbon-footprint_1kg
4345,7310100565808,Gluten free chocolate cookies,150 g,Cardboard,Semper,"Snacks,Sweet snacks,Biscuits and cakes,Biscuit...",NaN,No gluten,Finland,e,Sugary snacks,Biscuits and cakes,"Sugary snacks,Biscuits and cakes",NaN,NaN,Chocolate chip cookies,2.50
1921,6405096008031,Kebab-valkosipulipanini,235 g,NaN,Mr-panini,"Plant-based foods and beverages,Plant-based fo...",NaN,"High in salt - Finland,No lactose",Finland,d,Cereals and potatoes,Bread,"Cereals and potatoes,Bread",NaN,NaN,Paninis,2.50
2906,40893532,Cashew Nuts,500g,"Plastic,Bag,es:muovi,es:producto",alesto,"Plant-based foods and beverages,Plant-based fo...",fi:muu-kuin-eu,"Green Dot,Nutriscore,Nutriscore Grade B","Finland,France,Germany,Italy,Spain",b,Salty snacks,Nuts,"Salty snacks,Nuts",28.0,e,Cashew nuts,2.90
2989,8852018101055,yum yum Shrimps,60g,"Plastic,Pp-polypropylene,Bag,de:produkt",Yum-yum,"Plant-based foods and beverages,Plant-based fo...",Thailand,"Sustainable,Halal,Sustainable Palm Oil,Roundta...","Finland,France,Germany",unknown,Cereals and potatoes,Cereals,"Cereals and potatoes,Cereals",45.0,c,Dehydrated Asian-style soup with noodles,2.65
1494,4056489598190,Pinaatti Keitto,NaN,NaN,Kartanon,"Meals,Soups",NaN,NaN,Finland,unknown,Composite foods,One-dish meals,"Composite foods,One-dish meals",NaN,NaN,Soups,2.95


In [36]:
df_merged.to_csv("merged_data.csv")

In [37]:
df_main = pd.read_csv("merged_data.csv")
df_update = pd.read_csv("to_categories.csv")
 
df_main["code"] = df_main["code"].astype(str)
df_update["code"] = df_update["code"].astype(str)
 
df_main_updated = df_main.merge(df_update, on='code', how='left')
df_main_updated
 
# Update only where new categories exist
df_main_updated['pnns_groups_2'] = df_main_updated['new_category (pnns_groups_2)'].combine_first(df_main_updated['pnns_groups_2'])
 
# Drop the helper column
df_main_updated.drop(columns=['new_category (pnns_groups_2)'], inplace=True)
 
df_main_updated.to_csv("updated_products.csv", index=False)
 
df_main_updated.sample(3)

,Unnamed: 0,code,product_name,quantity,packaging_en,brands_en,categories_en,origins_en,labels_en,countries_en,nutriscore_grade,pnns_groups_1,pnns_groups_2,food_groups_en,environmental_score_score,environmental_score_grade,main_category_en,carbon-footprint_1kg
1382,1381,6430064404959,Sukkela suklaa,NaN,NaN,Pingviini,"Desserts,Frozen foods,Frozen desserts,Ice crea...",NaN,NaN,Finland,e,Milk and dairy products,Ice cream,"Milk and dairy products,Ice cream",NaN,NaN,Ice cream cones,2.60
3339,3337,20556372,Mojito mocktail alkoholiton,750 ml,"Plastic,Bottle",Solevita,"Beverages and beverages preparations,Beverages...",NaN,"No alcohol,fr:produits-faisant-reference-au-mo...","Finland,France",e,Beverages,Alcoholic beverages,"Beverages,Sweetened beverages",NaN,unknown,Sweetened beverages,0.55
1002,1002,6438460602412,Kauravälipala mustaherukka,400 g,NaN,Kotimaista,"Plant-based foods and beverages,Fermented food...",NaN,NaN,Finland,b,Sugary snacks,Sweets,"Sugary snacks,Sweets",NaN,NaN,Oat milk yogurts,2.95


In [38]:
df_main_updated['pnns_groups_2'].value_counts()

pnns_groups_2
Sweets                              343
One-dish meals                      292
Chocolate products                  237
Bread                               236
Biscuits and cakes                  234
Plant-based milk substitutes        221
Cheese                              201
Dressings and sauces                192
Yogurt                              188
Milk and yogurt                     188
Dairy desserts                      187
Breakfast cereals                   184
Protein snacks and drinks           157
Processed meat                      154
Cereals                             152
Ice cream                           132
Sweetened beverages                 120
Alcoholic beverages                  84
Fats                                 78
Poultry                              76
Vegetables                           71
Artificially sweetened beverages     70
Appetizers                           66
Pastries                             63
Fruit juices              